In [ ]:
import os

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64" 
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [913 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:14 

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-09-30 18:02:16--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.95MB/s    in 0.2s    

2022-09-30 18:02:16 (4.95 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
df.show()

In [ ]:
# Drop null values and count rows
df = df.dropna()
num_rows = df.count()
num_rows

960679

In [ ]:
# Check data types
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [ ]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

In [ ]:
# Transform dataset to fit tables in the schema file
cleaned_df = df.withColumn("review_date",col("review_date").cast("date"))

cleaned_df.printSchema()
cleaned_df.show(10)     

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_ratin

In [ ]:
# Create review_id DataFrame to match review_id table
review_df = cleaned_df.select(['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date'])
review_df.printSchema()
review_df.show()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3O9SGZBVQBV76|    3653882|B00FALQ1ZC|     937001370| 2015-08-31|
| RKH8BNC3L5DLF|   14661224|B00D3RGO20|     484010722| 2015-08-31|
|R2HLE8WKZSU3NL|   27324930|B00DKYC7TK|     361166390| 2015-08-31|
|R31U3UH5AZ42LL|    7211452|B000EQS1JW|     958035625| 2015-08-31|
|R2SV659OUJ945Y|   12733322|B00A6GFD7S|     765328221| 2015-08-31|
| RA51CP8TR5A2L|    6576411|B00EYSOSE8|     230493695| 2015-08-31|
| RB2Q7DLDN6TH6|   11811565|B00WM0QA3M|     549298279| 2015-08-31|
|R2RHFJV0UYBK3Y|   49401598|B00A4EYBR0|     844009113| 2015-08-31|
|R2Z6JOQ94LFHEP|   45925069|B00MAMPGGE|   

In [ ]:
review_df.count()

960679

In [ ]:
# Create products DataFrame to match products table
products_df = cleaned_df.select(['product_id', 'product_title'])
products_df = products_df.dropDuplicates()
products_df.printSchema()
products_df.show()

In [ ]:
products_df.count()

149694

In [ ]:
# Create customers DataFrame to match customers table
cust_df = cleaned_df.select(['customer_id'])
customers_df = cust_df.groupBy('customer_id').count()
customers_df = customers_df.withColumnRenamed('count', 'customer_count')
# Change customer_count data type from long to integer
customers_df = customers_df.withColumn("customer_count",col("customer_count").cast("int"))
# Double check Schema
customers_df.printSchema()

#Getting the all Id that appears more than once
customers_df.filter('customer_count > 1').select(['customer_id']).show(5)

In [ ]:
# Create a columns 'Duplicate_indicator' and after that make a filter
import pyspark.sql.functions as f
new_df = df.join(
    df.groupBy(df.columns).agg((f.count("customer_id")>1).cast("int").alias("Duplicate_indicator")),
    on=df.columns,
    how="inner"
)
#Just a Quick View to see how it gone
new_df.show(10)

# Making some consults


In [ ]:
df.groupBy('star_rating').count().orderBy('star_rating').withColumnRenamed('count', 'Quantitative').show(5)

+-----------+------------+
|star_rating|Quantitative|
+-----------+------------+
|          1|       85158|
|          2|       52032|
|          3|       79143|
|          4|      172853|
|          5|      571493|
+-----------+------------+



In [ ]:
# Pivoting Spark DataFrame
df.groupBy(['vine', 'star_rating']).count().groupBy('vine').pivot("star_rating").sum("count").show()

+----+-----+-----+-----+------+------+
|vine|    1|    2|    3|     4|     5|
+----+-----+-----+-----+------+------+
|   Y|   35|   87|  266|   754|   605|
|   N|85123|51945|78877|172099|570888|
+----+-----+-----+-----+------+------+



In [ ]:
import pyspark.sql.functions as F
df.groupBy("total_votes", "star_rating").agg(F.stddev("total_votes")).show(5)


+-----------+-----------+------------------------+
|total_votes|star_rating|stddev_samp(total_votes)|
+-----------+-----------+------------------------+
|         71|          4|                     0.0|
|        162|          3|                     NaN|
|         92|          2|                     NaN|
|        247|          5|                     NaN|
|         10|          2|                     0.0|
+-----------+-----------+------------------------+
only showing top 5 rows



In [ ]:
df.groupBy(['total_votes', 'star_rating']).count().show()

+-----------+-----------+-----+
|total_votes|star_rating|count|
+-----------+-----------+-----+
|         71|          4|    8|
|        162|          3|    1|
|         92|          2|    1|
|        247|          5|    1|
|         10|          2|  303|
|         62|          1|   13|
|         41|          3|   14|
|        118|          1|    1|
|         69|          1|    5|
|        104|          5|    8|
|        108|          1|    2|
|        105|          1|    3|
|        340|          5|    1|
|        185|          1|    1|
|         16|          3|  122|
|         34|          5|   80|
|         17|          1|  243|
|         17|          2|   88|
|        115|          5|    5|
|        491|          4|    1|
+-----------+-----------+-----+
only showing top 20 rows

